# Techniques of High Performance Computing - Assignment 2

### Name: John Duffy

### Student Number: 19154676

In [2]:
# Import libraries common to Questions 1 & 2.

import numpy as np
import pyopencl as cl

# Question 1

## OpenCL CSR Matrix-Vector Product

### Program Description

This program...

### Optimisation Results

A class LinearOperatorBaseline was created to use as a baseline for measuring the performance gains through subsequnt use of OpenCL and AVX2.

In [30]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import LinearOperator

In [33]:
# Define the class LinearOperatorBaseline for baseline performance measurements.

class LinearOperatorBaseline(LinearOperator):
    """
    This class... 
    """
    
    def __init__(self, data, indices, indptr):
        """
        """
        self.data = data
        self.indices = indices
        self.indptr = indptr
        self.shape = (len(indptr) - 1, len(indptr) - 1)  # Assume square.
    
    def _matvec(self, x):
        """
        """
        y = np.zeros(x.shape[0], dtype=np.float64)
        
        for i in range(self.shape[1]): 
            y[i] = np.dot(self.data[self.indptr[i]:self.indptr[i + 1]], x[self.indices[self.indptr[i]]:self.indptr[i + 1]])
            
        return y

In [34]:
# Create a test instance of LinearOperatorBaseline.

N = 10000

csr = csr_matrix(np.eye((N), dtype=np.float64))

linear_operator = LinearOperatorBaseline(csr.data, csr.indices, csr.indptr)

v = np.full((N), 2)

In [35]:
%%timeit

linear_operator.matvec(v)

22.8 ms ± 286 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%%timeit

linear_operator * v

22.9 ms ± 545 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
linear_operator.matvec(v)

array([2., 2., 2., ..., 2., 2., 2.])

In [25]:
linear_operator * v

array([2., 2., 2., ..., 2., 2., 2.])

# Question 2

## Solving a Poisson Problem with OpenCL

### Program Description

This program...

In [ ]:
from scipy.sparse.linalg import cg 